In [39]:
from feature.make_dataset import make_data
import pandas as pd
from glob import glob
from sklearn.metrics import mean_squared_error
from sklearn.model_selection import train_test_split
import glob
from xgboost import XGBRegressor
from catboost import CatBoostRegressor
from lightgbm import LGBMRegressor
from sklearn.linear_model import LinearRegression

In [40]:
train_input_list = './data/train_input/*.csv'
train_target_list = './data/train_target/*.csv'

test_input_list = './data/test_input/*.csv'
test_target_list = './data/test_target/*.csv'

In [41]:
train, test, p_train, p_test = make_data(train_input_list, train_target_list, test_input_list, test_target_list)

c:\mb_job\project\Competition\Lettuce-Growth-Environment-Prediction\feature\base_dataset.py:58: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df[cum_col][i*24:(i+1)*24] = cumsum
c:\mb_job\project\Competition\Lettuce-Growth-Environment-Prediction\feature\base_dataset.py:72: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df[cum_col][i*24:(i+1)*24] = cumsum


In [42]:
train_1 = pd.read_csv('./data/preprocessing/train_quantile1.csv')
test_1 = pd.read_csv('./data/preprocessing/test_quantile1.csv')
train_3 = pd.read_csv('./data/preprocessing/train_quantile3.csv')
test_3 = pd.read_csv('./data/preprocessing/test_quantile3.csv')
train_std = pd.read_csv('./data/preprocessing/train_std.csv')
test_std = pd.read_csv('./data/preprocessing/test_std.csv')

In [43]:
train_1 = train_1[['내부온도관측치', '내부습도관측치', 'co2관측치', 'ec관측치', '시간당분무량',
                  '일간누적분무량', '시간당백색광량', '일간누적백색광량', '시간당적색광량', '일간누적적색광량', '시간당청색광량',
                  '일간누적청색광량', '시간당총광량', '일간누적총광량']]
test_1 = test_1[['내부온도관측치', '내부습도관측치', 'co2관측치', 'ec관측치', '시간당분무량',
                  '일간누적분무량', '시간당백색광량', '일간누적백색광량', '시간당적색광량', '일간누적적색광량', '시간당청색광량',
                  '일간누적청색광량', '시간당총광량', '일간누적총광량']]
train_3 = train_3[['내부온도관측치', '내부습도관측치', 'co2관측치', 'ec관측치', '시간당분무량',
                  '일간누적분무량', '시간당백색광량', '일간누적백색광량', '시간당적색광량', '일간누적적색광량', '시간당청색광량',
                  '일간누적청색광량', '시간당총광량', '일간누적총광량']]
test_3 = test_3[['내부온도관측치', '내부습도관측치', 'co2관측치', 'ec관측치', '시간당분무량',
                  '일간누적분무량', '시간당백색광량', '일간누적백색광량', '시간당적색광량', '일간누적적색광량', '시간당청색광량',
                  '일간누적청색광량', '시간당총광량', '일간누적총광량']]
train_std = train_std[['내부온도관측치', '내부습도관측치', 'co2관측치', 'ec관측치', '시간당분무량',
                  '일간누적분무량', '시간당백색광량', '일간누적백색광량', '시간당적색광량', '일간누적적색광량', '시간당청색광량',
                  '일간누적청색광량', '시간당총광량', '일간누적총광량']]
test_std = test_std[['내부온도관측치', '내부습도관측치', 'co2관측치', 'ec관측치', '시간당분무량',
                  '일간누적분무량', '시간당백색광량', '일간누적백색광량', '시간당적색광량', '일간누적적색광량', '시간당청색광량',
                  '일간누적청색광량', '시간당총광량', '일간누적총광량']]


In [44]:
train_3.columns = [col+'3' for col in train_3.columns]
test_3.columns = [col+'3' for col in test_3.columns]
train_1.columns = [col+'1' for col in train_1.columns]
test_1.columns = [col+'1' for col in test_1.columns]
train_std.columns = [col+'std' for col in train_std.columns]
test_std.columns = [col+'std' for col in test_std.columns]

In [45]:
train = train.reset_index()
test  = test.reset_index()
train_3 = train_3.reset_index()
test_3 = test_3.reset_index()
train_1 = train_1.reset_index()
test_1 = test_1.reset_index()
train_std = train_std.reset_index()
test_std = test_std.reset_index()


In [46]:
train = pd.concat([train, train_1, train_3, train_std], axis=1)
test = pd.concat([test, test_1, test_3, test_std], axis=1)

In [47]:
train = train.drop(['index'], axis=1)
test = test.drop(['index'], axis=1)

In [48]:
def model_train(train, test, model):
    '''
    train shape = (784, 18), test shape = (140, 18)
    model => sklearn api model
    '''

    X = train.drop(['predicted_weight_g', 'Case', 'obs_time'], axis=1)
    y = train['predicted_weight_g']

    X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=0.3, random_state=113, shuffle=True)
    
    x_test = test.drop(['predicted_weight_g', 'Case', 'obs_time'], axis=1)
    
    model.fit(X_train, y_train)
    y_val_pred = model.predict(X_val)

    rmse = mean_squared_error(y_val, y_val_pred)**0.5
    print(f"validation rmse: {rmse}")
    
    x_train = train.drop(['predicted_weight_g', 'Case', 'obs_time'], axis=1)
    y_train = train['predicted_weight_g']
    
    model.fit(x_train, y_train)
    y_pred = model.predict(x_test)
    x_test['predicted_weight_g'] = y_pred
    submit = x_test[['DAT', 'predicted_weight_g']]
    submit['DAT'] = submit['DAT']+1
    all_target_list = sorted(glob.glob('./data/test_target/*.csv'))
    for idx, test_path in enumerate(all_target_list):
        submit_df = pd.read_csv(test_path)
        submit_df['predicted_weight_g'] = submit['predicted_weight_g'][idx*28:idx*28+28].values
        submit_df.to_csv(test_path, index=False)

In [31]:
xgb = XGBRegressor()
cat = CatBoostRegressor(verbose=100, early_stopping_rounds=50, task_type="GPU")
lgbm = LGBMRegressor()
linear = LinearRegression()

model_train(train, test, cat)

Learning rate set to 0.037333
0:	learn: 40.7823749	total: 14.5ms	remaining: 14.5s
100:	learn: 7.7160615	total: 1.39s	remaining: 12.3s
200:	learn: 5.4069313	total: 2.73s	remaining: 10.8s
300:	learn: 4.9669458	total: 4.02s	remaining: 9.33s
400:	learn: 4.5432955	total: 5.32s	remaining: 7.95s
500:	learn: 4.2288732	total: 6.66s	remaining: 6.63s
600:	learn: 3.9639934	total: 7.93s	remaining: 5.27s
700:	learn: 3.8610017	total: 9.27s	remaining: 3.95s
800:	learn: 3.7032992	total: 10.6s	remaining: 2.63s
900:	learn: 3.5861392	total: 11.9s	remaining: 1.3s
999:	learn: 3.4773178	total: 13.1s	remaining: 0us
validation rmse: 8.798115197165734
Learning rate set to 0.039126
0:	learn: 40.3656518	total: 13.2ms	remaining: 13.2s
100:	learn: 7.2823253	total: 1.41s	remaining: 12.6s
200:	learn: 5.5371537	total: 2.77s	remaining: 11s
300:	learn: 5.1713181	total: 4.1s	remaining: 9.53s
400:	learn: 4.8992119	total: 5.41s	remaining: 8.08s
500:	learn: 4.5806003	total: 6.78s	remaining: 6.75s
600:	learn: 4.3889438	total

C:\Users\jkpark\AppData\Local\Temp\ipykernel_31256\2960248603.py:30: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  submit['DAT'] = submit['DAT']+1
